In [3]:
import pandas as pd
# 5-fold split
data = '../annotation/annotated_cases.csv'
outdir = '../datasets/'
df = pd.read_csv(data).fillna(0)
df

,pmc_s,text,N_male,N_female,SBP_in_male_mean,SBP_in_female_mean,SBP_in_male_std,SBP_in_female_std,DBP_in_male_mean,DBP_in_female_mean,DBP_in_male_std,DBP_in_female_std,positive_case
0,PMC4685985,A post hoc comparison of gender revealed mal...,3.0,3.0,125.00,116.90,8.60,15.50,69.80,65.40,9.50,10.40,1
1,PMC9554358,When we compared males with obesity and female...,1.0,1.0,127.76,117.68,10.93,11.66,78.80,85.12,7.55,7.07,1
2,PMC6943842,Observed values of the decreased systolic and...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
3,PMC8062496,Clinical and laboratory characteristics of non...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
4,PMC4036422,Hemodynamic data All subjects Males Females ...,3.0,3.0,121.00,112.00,2.00,3.00,64.00,63.00,2.00,2.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,PMC3257590,Baseline Characteristics of Enrolled Subjects ...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
209,PMC3490738,Descriptive data for female and male adolescen...,1631.0,1813.0,127.10,117.30,15.60,12.80,68.70,69.20,10.50,10.30,1
210,PMC8359777,"To confirm hypertension in SHR, blood pressure...",1.0,1.0,215.00,182.00,5.00,4.00,178.00,143.00,7.00,3.00,1
211,PMC3714674,Clinical and echocardiographic parameters of n...,2.0,2.0,115.44,105.07,24.36,25.22,73.75,70.76,8.89,7.42,1


In [4]:
# K-fold split
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import os, sys

N = 5

df = pd.read_csv(data)
skf = KFold(n_splits=N, random_state=1, shuffle=True)

for i, (train_index, test_index) in enumerate(skf.split(df)):
    xTrain = df.loc[train_index, :]
    xTest = df.loc[test_index, :]
    new_xTrain, xDev = train_test_split(xTrain, test_size=0.2, random_state=1, shuffle=True)

    print('train:{} dev:{} test:{}'.format(len(new_xTrain), len(xDev), len(xTest)))

    split_path = '{}/data_splits_{}'.format(outdir, i+1)
    if not os.path.exists(split_path):
        os.makedirs(split_path)

    new_xTrain.to_csv(f'{split_path}/train.csv', index=False)
    xDev.to_csv(f'{split_path}/dev.csv', index=False)
    xTest.to_csv(f'{split_path}/test.csv', index=False)



train:136 dev:34 test:43
train:136 dev:34 test:43
train:136 dev:34 test:43
train:136 dev:35 test:42
train:136 dev:35 test:42


# Prepare BIO data

In [5]:
# Convert the data into the BIO format
from nltk.tokenize import wordpunct_tokenize, sent_tokenize

targets = [
'N_male',
'N_female',
'SBP_in_male_mean',
'SBP_in_female_mean',
'SBP_in_male_std',
'SBP_in_female_std',
'DBP_in_male_mean',
'DBP_in_female_mean',
'DBP_in_male_std',
'DBP_in_female_std'
]

def list_match(a, b):
    assert len(a) == len(b)
    for i in range(len(a)):
        if a[i] != b[i]:
            return False
    return True

def find_target(long_list, short_list):
    for i in range(len(long_list)-len(short_list)):
        same_li = list_match(long_list[i:i+len(short_list)], short_list)
        if same_li:
            return i
    return -1
    
for i in range(N):
    for dataset in ['train', 'dev', 'test']:
        data_file = f'{outdir}/data_splits_{i+1}/{dataset}.csv'
        
        bio_outdir = f'{outdir}/data_splits_BIO_{i+1}/'
        if not os.path.exists(bio_outdir):
            os.makedirs(bio_outdir)
        out_file = f'{bio_outdir}/{dataset}.csv'

        fw = open(out_file, 'w')        
        df = pd.read_csv(data_file)
        for _, row in df.iterrows():
            label = row['positive_case']
            text = row['text'].strip()
            sentences = sent_tokenize(text)
            all_tokens = [wordpunct_tokenize(sentence) for sentence in sentences]            
            all_target_tokens = [wordpunct_tokenize(str(row[target]).replace('.0', '')) for target in targets]
            
            if label:
#                 print(row)
                match_count = 0
                for tokens in all_tokens:
                    pos_map = {}
                    for k, target_tokens in enumerate(all_target_tokens):
#                         print(tokens)
#                         print(target_tokens)
                        start_pos = find_target(tokens, target_tokens)
                        if start_pos >= 0:
#                             print(target_tokens)
                            match_count += 1
                            for pos in range(start_pos, start_pos+len(target_tokens)):
                                if pos == start_pos:
                                    pos_map[pos] = 'B-' + targets[k]
                                else:
                                    pos_map[pos] = 'I-' + targets[k]

                    for k, token in enumerate(tokens):
                        if k in pos_map:
                            fw.write(f'{token} {pos_map[k]}\n')
                        else:
                            fw.write(f'{token} O\n')
                    fw.write('\n')
#                 print('Match count', match_count)
                
            else:
                for tokens in all_tokens:
                    for token in tokens:
                        fw.write(f'{token} O\n')
                    fw.write('\n')

        fw.close()
                        